In [1]:
import random
from faker import Faker

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, year, month, dayofmonth

### Spark Setup

In [7]:
spark_jar_packages = ",".join([
    "org.apache.hudi:hudi-spark3.5-bundle_2.12:1.0.0",
    "org.apache.spark:spark-hive_2.12:3.5.3",
    "com.google.code.findbugs:jsr305:3.0.2",
    #"org.apache.thrift:libthrift:0.14.0",
    # "org.slf4j:slf4j-api:1.7.36",
    # "org.apache.logging.log4j:log4j-slf4j-impl:2.24.3",
    #"org.apache.hive:hive-metastore:2.3.9",
    #"org.apache.hive:hive-exec:2.3.9",
    # "org.apache.hadoop:hadoop-aws:3.3.4",
    # "com.amazonaws:aws-java-sdk-bundle:1.12.262",
])

In [8]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("HudiWithHive") \
    .master("local[*]")  \
    .config("spark.jars.packages", spark_jar_packages)  \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.sql.catalogImplementation", "hive") \
    .config("spark.hadoop.hive.metastore.uris", "thrift://localhost:9083") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.kryo.registrator", "org.apache.spark.HoodieSparkKryoRegistrar") \
    .config("spark.sql.extensions", "org.apache.spark.sql.hudi.HoodieSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.hudi.catalog.HoodieCatalog") \
    .enableHiveSupport() \
    .getOrCreate()


25/01/10 12:21:42 WARN Utils: Your hostname, baptvit resolves to a loopback address: 127.0.1.1; using 192.168.2.129 instead (on interface wlp4s0)
25/01/10 12:21:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /home/baptvit/.ivy2/cache
The jars for the packages stored in: /home/baptvit/.ivy2/jars
org.apache.hudi#hudi-spark3.5-bundle_2.12 added as a dependency
org.apache.spark#spark-hive_2.12 added as a dependency
com.google.code.findbugs#jsr305 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e36ae98c-2d53-471f-b04d-8fc35203c0a6;1.0
	confs: [default]


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.hudi#hudi-spark3.5-bundle_2.12;1.0.0 in central
	found org.apache.hive#hive-storage-api;2.8.1 in central
	found org.slf4j#slf4j-api;1.7.36 in central
	found org.apache.spark#spark-hive_2.12;3.5.3 in central
	found org.apache.hive#hive-common;2.3.9 in central
	found commons-cli#commons-cli;1.5.0 in central
	found commons-lang#commons-lang;2.6 in local-m2-cache
	found org.apache.commons#commons-lang3;3.12.0 in central
	found jline#jline;2.14.6 in central
	found joda-time#joda-time;2.12.5 in central
	found org.apache.commons#commons-compress;1.23.0 in central
	found com.tdunning#json;1.8 in local-m2-cache
	found io.dropwizard.metrics#metrics-core;4.2.19 in central
	found io.dropwizard.metrics#metrics-jvm;4.2.19 in central
	found io.dropwizard.metrics#metrics-json;4.2.19 in central
	found com.fasterxml.jackson.core#jackson-core;2.15.2 in central
	found com.fasterxml.jackson.core#jackson-databind;2.15.2 in central
	found com.fasterxml.jackson.core#jackson-annotations;2.15.

In [9]:
spark

In [20]:
spark.sql("SHOW DATABASES;").show()

+---------+
|namespace|
+---------+
|  default|
+---------+



In [14]:
# Define table name and base path
database_name = "default"
table_name = "hudi_cow"
base_path = "file:///home/baptvit/Documents/github/lakehouse-labs/ranger/path/to/hudi/table"

# Sample data
data = [
    ("row_1", "2021/01/01", 0, "bob", "v_0", "toBeDel0", 0, 1000000),
    ("row_2", "2021/01/01", 0, "john", "v_0", "toBeDel0", 0, 1000000),
    ("row_3", "2021/01/02", 0, "tom", "v_0", "toBeDel0", 0, 1000000)
]

In [15]:
# Define schema
from pyspark.sql.types import StructType, StructField, StringType, LongType, IntegerType

schema = StructType([
    StructField("rowId", StringType(), True),
    StructField("partitionId", StringType(), True),
    StructField("preComb", LongType(), True),
    StructField("name", StringType(), True),
    StructField("versionId", StringType(), True),
    StructField("toBeDeletedStr", StringType(), True),
    StructField("intToLong", IntegerType(), True),
    StructField("longToInt", LongType(), True)
])

# Create DataFrame
df = spark.createDataFrame(data, schema)

In [16]:
df.show()

+-----+-----------+-------+----+---------+--------------+---------+---------+
|rowId|partitionId|preComb|name|versionId|toBeDeletedStr|intToLong|longToInt|
+-----+-----------+-------+----+---------+--------------+---------+---------+
|row_1| 2021/01/01|      0| bob|      v_0|      toBeDel0|        0|  1000000|
|row_2| 2021/01/01|      0|john|      v_0|      toBeDel0|        0|  1000000|
|row_3| 2021/01/02|      0| tom|      v_0|      toBeDel0|        0|  1000000|
+-----+-----------+-------+----+---------+--------------+---------+---------+



In [19]:
# Write data as a Hudi table
df.write.format("hudi") \
    .option("hoodie.datasource.write.precombine.field", "preComb") \
    .option("hoodie.datasource.write.recordkey.field", "rowId") \
    .option("hoodie.datasource.write.partitionpath.field", "partitionId") \
    .option("hoodie.database.name", database_name) \
    .option("hoodie.table.name", table_name) \
    .option("hoodie.datasource.write.table.type", "COPY_ON_WRITE") \
    .option("hoodie.datasource.write.operation", "upsert") \
    .option("hoodie.datasource.write.hive_style_partitioning", "true") \
    .option("hoodie.datasource.hive_sync.enable", "true") \
    .option("hoodie.datasource.hive_sync.mode", "jdbc") \
    .option("hoodie.datasource.hive_sync.jdbcurl", "jdbc:hive2://localhost:10000") \
    .option("hoodie.datasource.hive_sync.username", "hive") \
    .option("hoodie.datasource.hive_sync.password", "rangerR0cks!") \
    .mode("overwrite") \
    .save(base_path)

Py4JJavaError: An error occurred while calling o154.save.
: org.apache.hudi.exception.HoodieMetaSyncException: Could not sync using the meta sync class org.apache.hudi.hive.HiveSyncTool
	at org.apache.hudi.sync.common.util.SyncUtilHelpers.runHoodieMetaSync(SyncUtilHelpers.java:106)
	at org.apache.hudi.sync.common.util.SyncUtilHelpers.runHoodieMetaSync(SyncUtilHelpers.java:72)
	at org.apache.hudi.HoodieSparkSqlWriterInternal.$anonfun$metaSync$2(HoodieSparkSqlWriter.scala:925)
	at scala.collection.mutable.HashSet.foreach(HashSet.scala:79)
	at org.apache.hudi.HoodieSparkSqlWriterInternal.metaSync(HoodieSparkSqlWriter.scala:923)
	at org.apache.hudi.HoodieSparkSqlWriterInternal.commitAndPerformPostOperations(HoodieSparkSqlWriter.scala:1022)
	at org.apache.hudi.HoodieSparkSqlWriterInternal.writeInternal(HoodieSparkSqlWriter.scala:534)
	at org.apache.hudi.HoodieSparkSqlWriterInternal.$anonfun$write$1(HoodieSparkSqlWriter.scala:190)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$wit

In [ ]:
from pyspark.sql.functions import current_timestamp

# Load a dataset
df = spark.read.csv("industry.csv", header=True, inferSchema=True)
df_with_ts = df.withColumn("ts", current_timestamp())

In [ ]:
df_with_ts.show()

In [ ]:
table_name = "hudi_table"
database_name = "default"
base_path = "file:///home/baptvit/Documents/github/lakehouse-labs/ranger/path/to/hudi/table"

In [ ]:
# Write data as a Hudi table
df.write.format("hudi") \
    .option("hoodie.table.name", table_name) \
    .option("hoodie.datasource.write.recordkey.field", "Industry") \
    .option("hoodie.datasource.write.table.name", table_name) \
    .option("hoodie.datasource.write.operation", "upsert") \
    .option("hoodie.datasource.write.precombine.field", "Industry") \
    .option("hoodie.datasource.write.hive_style_partitioning", "true") \
    .option("hoodie.datasource.hive_sync.enable", "true") \
    .option("hoodie.database.name", database_name) \
    .option("hoodie.datasource.hive_sync.database", database_name) \
    .option("hoodie.datasource.hive_sync.table", table_name) \
    .mode("overwrite") \
    .save(base_path)

In [ ]:
# Write data as a Hudi table
df.write.format("hudi") \
    .option("hoodie.table.name", table_name) \
    .option("hoodie.datasource.write.recordkey.field", "Industry") \
    .option("hoodie.datasource.write.table.name", table_name) \
    .option("hoodie.datasource.write.operation", "upsert") \
    .option("hoodie.datasource.write.precombine.field", "Industry") \
    .option("hoodie.datasource.write.hive_style_partitioning", "true") \
    .option("hoodie.datasource.hive_sync.partition_fields", "ts") \
    .option("hoodie.datasource.hive_sync.enable", "true") \
    .option("hoodie.database.name", database_name) \
    .option("hoodie.datasource.hive_sync.database", database_name) \
    .option("hoodie.datasource.hive_sync.table", table_name) \
    .option("hoodie.datasource.hive_sync.jdbcurl", "jdbc:hive2://ranger-hive:10000/default") \
    .option("hoodie.datasource.hive_sync.username", "hive") \
    .option("hoodie.datasource.hive_sync.password", "rangerR0cks!") \
    .mode("overwrite") \
    .save(base_path)

In [ ]:
df_with_ts.write.format("hudi") \
    .options(**hudi_configs) \
    .mode("append")\
    .save("file:///home/baptvit/Documents/github/lakehouse-labs/ranger/path/to/hudi/table")

### Dataset Generation

In [ ]:
def generate_entry(faker: Faker, country_codes: list):
    return {
        "id": faker.unique.uuid4(),
        "name":  faker.name(),
        "email": faker.email(),
        "passport": faker.passport_number(),
        "country_code": random.choice(country_codes),
        "iban": faker.iban(),
        "swift": faker.swift11(),
        "created_at": faker.past_date(start_date='-90d').strftime('%Y-%m-%d')
    }

In [ ]:
def generate_dataset(num: int, seed: int):
    country_codes = ['US', 'CA', 'JP', 'KR', 'FR', 'GE', 'UK', 'BR', 'AR']
    Faker.seed(seed)
    faker = Faker()
    return [generate_entry(faker, country_codes) for _ in range(num)]

In [ ]:
dataset = generate_dataset(num=100_000, seed=739)

In [ ]:
df = spark.createDataFrame(dataset)\
        .withColumn("year", year(col("created_at")))\
        .withColumn("month", month(col("created_at")))\
        .withColumn("day", dayofmonth(col("created_at")))

### Hudi-Hive Integration

In [ ]:
df.write.format("hudi") \
    .option("hoodie.database.name", "hudi_raw") \
    .option("hoodie.table.name", "accounts") \
    .option("hoodie.datasource.write.recordkey.field", "id") \
    .option("hoodie.datasource.write.precombine.field", "created_at") \
    .option("hoodie.datasource.write.table.type", "COPY_ON_WRITE") \
    .option("hoodie.datasource.write.operation", "upsert") \
    .option("hoodie.datasource.meta.sync.enable", "true") \
    .option("hoodie.datasource.hive_sync.mode", "hms") \
    .option("hoodie.datasource.hive_sync.metastore.uris", "thrift://localhost:9083") \
    .option("hoodie.datasource.hive_sync.partition_fields", "year,month") \
    .option("hoodie.datasource.hive_sync.partition_extractor_class", "org.apache.hudi.hive.MultiPartKeysValueExtractor") \
    .option("hoodie.datasource.write.hive_style_partitioning","true") \
    .partitionBy("year", "month") \
    .mode("overwrite") \
    .save("s3a://lakehouse-raw/hudi/accounts")

### Upsert Dataset

In [ ]:
entries = [
    # Existing entries
    dataset[2], 
    dataset[4], 
    dataset[7],
    dataset[11],
    # New entries
    *generate_dataset(4, seed=1037)
]

In [ ]:
for entry in entries:
    username = entry['name'].lower().replace(" ", ".")
    entry['email'] = f"{username}@domain.com"

In [ ]:
upsert_df = spark.createDataFrame(entries)\
        .withColumn("year", year(col("created_at")))\
        .withColumn("month", month(col("created_at")))\
        .withColumn("day", dayofmonth(col("created_at")))

In [ ]:
upsert_df.show(8, truncate=False)

In [ ]:
upsert_df.createOrReplaceTempView("upsert_data")

### Upsert Strategy

In [ ]:
upsert_df.write.format("hudi") \
    .option("hoodie.database.name", "hudi_raw") \
    .option("hoodie.table.name", "accounts") \
    .option("hoodie.datasource.write.recordkey.field", "id") \
    .option("hoodie.datasource.write.precombine.field", "created_at") \
    .option("hoodie.datasource.write.table.type", "COPY_ON_WRITE") \
    .option("hoodie.datasource.write.operation", "upsert") \
    .option("hoodie.datasource.meta.sync.enable", "true") \
    .option("hoodie.datasource.hive_sync.mode", "hms") \
    .option("hoodie.datasource.hive_sync.metastore.uris", "thrift://localhost:9083") \
    .option("hoodie.datasource.hive_sync.partition_fields", "year,month") \
    .option("hoodie.datasource.hive_sync.partition_extractor_class", "org.apache.hudi.hive.MultiPartKeysValueExtractor") \
    .option("hoodie.datasource.write.hive_style_partitioning","true") \
    .partitionBy("year", "month") \
    .mode("append") \
    .save("s3a://lakehouse-raw/hudi/accounts")

### Hudi Metadata (WIP)

In [ ]:
spark.sql("""
    call show_commits (
        table => 'hudi_raw.accounts',
        from_commit => '0'
    )    
""").show(truncate=False)

In [ ]:
hudi_changes = spark.read.format("hudi") \
    .option("hoodie.datasource.query.type", "incremental") \
    .option("hoodie.datasource.read.begin.instanttime", 0) \
    .load("s3a://lakehouse-raw/hudi/accounts")

In [ ]:
hudi_changes.show()